<a href="https://colab.research.google.com/github/rafaelpivetta/tech-challenge-fase3/blob/main/LoRA%20for%20Fine-Tuning%20Llama3%20LLMs-Revisado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
!pip install datasets peft trl bitsandbytes

In [20]:
import os
import torch
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, pipeline, logging
from peft import LoraConfig
from trl import SFTTrainer

base_model_name = "TinyLlama/TinyLlama-1.1B-intermediate-step-1431k-3T"
amazon_products_dataset = "ckandemir/amazon-products"
new_model ="llama-1.1B-chat-amazon-products"

dataset = load_dataset(amazon_products_dataset, split="train")
model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    device_map="auto",
)

model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(base_model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # pad sequences
tokenizer.padding_side = "right" # right pad sequences

In [21]:
# run inference
logging.set_verbosity(logging.CRITICAL)
prompt = "Describe the product Hot Wheels 2019 Advent Calendar Vehicles."
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)
result = pipe(f"{prompt}")
print(result[0]["generated_text"])
#

Describe the product Hot Wheels 2019 Advent Calendar Vehicles.
The 2019 Hot Wheels Advent Calendar is a 36-piece set of 1:64 scale die-cast cars. The set includes 12 cars, each with a different theme. The cars are all based on the 2019 Hot Wheels lineup, and include cars from the 2019 Monster Jam, Monster


In [22]:
#LoRA
peft_params = LoraConfig(
    lora_alpha=16, # multiplier of Lora outpul when its added to the full forward output
    lora_dropout=0.1, # with a probability of 10$ it will set random Lora output to 0
    r=32, # rank of Lora so matrices will have either LHS or RHS dimension of 64
    bias="none", # no bias term
    task_type="CAUSAL_LM"
)

training_params = TrainingArguments(output_dir='./results',
    num_train_epochs=3, # One pass over the dataset
    per_device_train_batch_size=2, # mbs=1
    gradient_accumulation_steps=16, # effective batch size 16*2
    optim="adamw_torch",
    save_steps=200, # checkpoint every 100 steps
    logging_steps=100,
    learning_rate=3e-4, # step size in the optimizer update
    weight_decay=0.001,
    fp16=True, # 16 bit
    bf16=False, # not supported on V100
    max_grad_norm=0.3, # gradient clipping improves convergence
    max_steps=-1,
    warmup_ratio=0.03, # learning rate warmup
    group_by_length=True,
    lr_scheduler_type="cosine", #cosine lr scheduler
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_params, # parameter efficient fine tuning AKA Lora
    dataset_text_field="Description",
    max_seq_length=128,
    tokenizer=tokenizer,
    args=training_params,
    packing=False
)

import gc # garbage collection
gc.collect()
torch.cuda.empty_cache() #clean cache

trainer.train() #train the model
trainer.model.save_pretrained(new_model)
trainer.tokenizer.save_pretrained(new_model)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_deprecation.py:100: FutureWarning: Deprecated argument(s) used in '__init__': dataset_text_field, max_seq_length. Will not be supported from version '1.0.0'.

Deprecated positional argument(s) used in SFTTrainer, please use the SFTConfig to set these arguments instead.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:321: UserWarning: You passed a `dataset_text_field` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(


Map:   0%|          | 0/23993 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:488: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


{'loss': 3.4557, 'grad_norm': 4.175568103790283, 'learning_rate': 0.0003997871820746685, 'epoch': 0.13336667500208385}
{'loss': 3.096, 'grad_norm': 3.4048638343811035, 'learning_rate': 0.0003963890460321148, 'epoch': 0.2667333500041677}
{'loss': 3.0291, 'grad_norm': 3.191901922225952, 'learning_rate': 0.00038891569880299245, 'epoch': 0.4001000250062516}
{'loss': 2.9489, 'grad_norm': 2.847163200378418, 'learning_rate': 0.00037765484607478077, 'epoch': 0.5334667000083354}
{'loss': 2.9003, 'grad_norm': 2.607309103012085, 'learning_rate': 0.0003627807660464467, 'epoch': 0.6668333750104193}
{'loss': 2.8526, 'grad_norm': 2.3783931732177734, 'learning_rate': 0.0003443966111625041, 'epoch': 0.8002000500125032}
{'loss': 2.7885, 'grad_norm': 2.2725889682769775, 'learning_rate': 0.000323016124777385, 'epoch': 0.9335667250145869}
{'loss': 2.6918, 'grad_norm': 1.876269817352295, 'learning_rate': 0.00029908296705977654, 'epoch': 1.0669334000166708}
{'loss': 2.63, 'grad_norm': 1.8575735092163086, 'le

('llama-1.1B-chat-amazon-products/tokenizer_config.json',
 'llama-1.1B-chat-amazon-products/special_tokens_map.json',
 'llama-1.1B-chat-amazon-products/tokenizer.model',
 'llama-1.1B-chat-amazon-products/added_tokens.json',
 'llama-1.1B-chat-amazon-products/tokenizer.json')

In [27]:
prompt = "Describe the product Hot Wheels 2019 Advent Calendar Vehicles."
pipe = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=100
)
result_fine_tuning = pipe(f'{prompt}')
print(result_fine_tuning[0]['generated_text'])

Describe the product Hot Wheels 2019 Advent Calendar Vehicles.
The Hot Wheels 2019 Advent Calendar Vehicles is a great gift for kids aged 8 and up. The Hot Wheels 2019 Advent Calendar Vehicles is a great gift for kids aged 8 and up. The Hot Wheels 2019 Advent Calendar Vehicles is a great gift for kids aged 8 and up
